In [5]:
import hashlib
import struct
import socket
import time
import random
from typing import List, Dict, Tuple, Optional

class BitcoinProtocol:
    """Implementación del protocolo Bitcoin P2P con handshake completo"""

    # Magic numbers por red
    MAGIC_MAINNET = bytes.fromhex('f9beb4d9')
    MAGIC_TESTNET = bytes.fromhex('0b110907')

    # Versión del protocolo
    PROTOCOL_VERSION = 70015

    # Servicios
    NODE_NETWORK = 1
    NODE_WITNESS = 8

    GENESIS_HASH = "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"

    def __init__(self, ip: str, port: int = 8333, timeout: int = 30, network: str = 'mainnet'):
        self.ip = ip
        self.port = port
        self.timeout = timeout
        self.network = network
        self.magic = self.MAGIC_MAINNET if network == 'mainnet' else self.MAGIC_TESTNET
        self.sock: Optional[socket.socket] = None
        self.connected = False
        self.handshake_complete = False

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        """Doble SHA256"""
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def checksum(payload: bytes) -> bytes:
        """Calcula el checksum del payload (primeros 4 bytes del doble SHA256)"""
        return BitcoinProtocol.double_sha256(payload)[:4]

    @staticmethod
    def create_varstr(s: str) -> bytes:
        """Crea un variable string con su longitud"""
        encoded = s.encode('utf-8')
        return BitcoinProtocol.create_varint(len(encoded)) + encoded

    @staticmethod
    def create_varint(n: int) -> bytes:
        """Crea un varint"""
        if n < 0xfd:
            return struct.pack('<B', n)
        elif n <= 0xffff:
            return b'\xfd' + struct.pack('<H', n)
        elif n <= 0xffffffff:
            return b'\xfe' + struct.pack('<I', n)
        else:
            return b'\xff' + struct.pack('<Q', n)

    @staticmethod
    def read_varint(data: bytes, offset: int = 0) -> Tuple[int, int]:
        """Lee un varint y retorna (valor, nuevo_offset)"""
        first_byte = data[offset]
        if first_byte < 0xfd:
            return first_byte, offset + 1
        elif first_byte == 0xfd:
            return struct.unpack('<H', data[offset+1:offset+3])[0], offset + 3
        elif first_byte == 0xfe:
            return struct.unpack('<I', data[offset+1:offset+5])[0], offset + 5
        else:
            return struct.unpack('<Q', data[offset+1:offset+9])[0], offset + 9

    def create_message(self, command: str, payload: bytes = b'') -> bytes:
        """Crea un mensaje del protocolo Bitcoin"""
        # Magic (4 bytes)
        msg = self.magic

        # Command (12 bytes, null-padded)
        cmd = command.encode('ascii')
        msg += cmd + b'\x00' * (12 - len(cmd))

        # Payload length (4 bytes, little-endian)
        msg += struct.pack('<I', len(payload))

        # Checksum (4 bytes)
        msg += self.checksum(payload)

        # Payload
        msg += payload

        return msg

    def create_network_address(self, ip: str, port: int, services: int = 0) -> bytes:
        """Crea una dirección de red (26 bytes sin timestamp)"""
        addr = struct.pack('<Q', services)  # Services (8 bytes)

        # IPv4-mapped IPv6 address
        addr += b'\x00' * 10 + b'\xff\xff'

        # IPv4 address
        addr += socket.inet_aton(ip)

        # Port (big-endian)
        addr += struct.pack('>H', port)

        return addr

    def create_version_payload(self) -> bytes:
        """Crea el payload del mensaje version"""
        payload = b''

        # Protocol version (4 bytes)
        payload += struct.pack('<I', self.PROTOCOL_VERSION)

        # Services (8 bytes)
        services = self.NODE_NETWORK
        payload += struct.pack('<Q', services)

        # Timestamp (8 bytes)
        payload += struct.pack('<q', int(time.time()))

        # Addr_recv (26 bytes) - dirección del nodo remoto
        payload += self.create_network_address(self.ip, self.port, services)

        # Addr_from (26 bytes) - nuestra dirección
        payload += self.create_network_address('0.0.0.0', 0, services)

        # Nonce (8 bytes)
        payload += struct.pack('<Q', random.getrandbits(64))

        # User agent
        user_agent = "/Kaoru:0.1.0/"
        payload += self.create_varstr(user_agent)

        # Start height (4 bytes)
        payload += struct.pack('<I', 0)

        # Relay (1 byte) - BIP37
        payload += struct.pack('<?', True)

        return payload

    def parse_message(self, data: bytes) -> Tuple[Optional[str], Optional[bytes], int]:
        """
        Parsea un mensaje del protocolo Bitcoin
        Retorna: (command, payload, bytes_consumed) o (None, None, 0) si hay error
        """
        if len(data) < 24:  # Mínimo: magic(4) + command(12) + length(4) + checksum(4)
            return None, None, 0

        # Verificar magic
        if data[:4] != self.magic:
            print(f"⚠️  Magic incorrecto: {data[:4].hex()} (esperado: {self.magic.hex()})")
            return None, None, 0

        # Extraer command
        command = data[4:16].rstrip(b'\x00').decode('ascii')

        # Extraer longitud del payload
        payload_length = struct.unpack('<I', data[16:20])[0]

        # Verificar si tenemos suficientes datos
        total_length = 24 + payload_length
        if len(data) < total_length:
            return None, None, 0

        # Extraer checksum
        expected_checksum = data[20:24]

        # Extraer payload
        payload = data[24:24 + payload_length]

        # Verificar checksum
        actual_checksum = self.checksum(payload)
        if expected_checksum != actual_checksum:
            print(f"⚠️  Checksum incorrecto para {command}")
            return None, None, total_length

        return command, payload, total_length

    def parse_version_payload(self, payload: bytes) -> Dict:
        """Parsea el payload de un mensaje version"""
        info = {}
        offset = 0

        info['version'] = struct.unpack('<I', payload[offset:offset+4])[0]
        offset += 4

        info['services'] = struct.unpack('<Q', payload[offset:offset+8])[0]
        offset += 8

        info['timestamp'] = struct.unpack('<q', payload[offset:offset+8])[0]
        offset += 8

        # Skip addr_recv (26 bytes)
        offset += 26

        # Skip addr_from (26 bytes)
        offset += 26

        # Nonce
        info['nonce'] = struct.unpack('<Q', payload[offset:offset+8])[0]
        offset += 8

        # User agent
        if offset < len(payload):
            ua_length, offset = self.read_varint(payload, offset)
            if offset + ua_length <= len(payload):
                info['user_agent'] = payload[offset:offset+ua_length].decode('utf-8', errors='ignore')
                offset += ua_length

        # Start height
        if offset + 4 <= len(payload):
            info['start_height'] = struct.unpack('<I', payload[offset:offset+4])[0]

        return info

    def connect(self) -> bool:
        """Establece conexión TCP con el nodo"""
        try:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.settimeout(self.timeout)

            print(f"🔌 Conectando a {self.ip}:{self.port}...")
            self.sock.connect((self.ip, self.port))

            self.connected = True
            print(f"✅ Conexión TCP establecida")
            return True

        except socket.timeout:
            print(f"❌ Timeout al conectar")
            return False
        except ConnectionRefusedError:
            print(f"❌ Conexión rechazada")
            return False
        except Exception as e:
            print(f"❌ Error de conexión: {e}")
            return False

    def send_message(self, command: str, payload: bytes = b'') -> bool:
        """Envía un mensaje al nodo"""
        if not self.connected or not self.sock:
            print("❌ No conectado")
            return False

        try:
            message = self.create_message(command, payload)
            self.sock.sendall(message)
            print(f"📤 Enviado: {command} ({len(payload)} bytes payload)")
            return True
        except Exception as e:
            print(f"❌ Error enviando {command}: {e}")
            return False

    def receive_messages(self, timeout: Optional[int] = None) -> List[Tuple[str, bytes]]:
        """Recibe mensajes del nodo con timeout extendido"""
        if not self.connected or not self.sock:
            return []

        messages = []
        buffer = b''
        original_timeout = self.sock.gettimeout()

        if timeout:
            self.sock.settimeout(timeout)

        try:
            while True:
                try:
                    chunk = self.sock.recv(8192)
                    if not chunk:
                        break
                    buffer += chunk

                    # Intentar parsear mensajes del buffer
                    while True:
                        command, payload, consumed = self.parse_message(buffer)
                        if command is None:
                            break

                        messages.append((command, payload))
                        print(f"📥 Recibido: {command} ({len(payload) if payload else 0} bytes)")
                        buffer = buffer[consumed:]

                    # Si ya tenemos suficientes mensajes para el handshake, salir
                    commands_received = [m[0] for m in messages]
                    if 'version' in commands_received and 'verack' in commands_received:
                        break

                except socket.timeout:
                    if messages:
                        break
                    continue

        except Exception as e:
            print(f"⚠️  Error recibiendo: {e}")
        finally:
            self.sock.settimeout(original_timeout)

        return messages

    def handshake(self) -> bool:
        """Realiza el handshake completo del protocolo Bitcoin"""
        print("\n" + "="*60)
        print("🤝 INICIANDO HANDSHAKE BITCOIN")
        print("="*60)

        # Paso 1: Enviar version
        print("\n📋 Paso 1: Enviando mensaje VERSION...")
        version_payload = self.create_version_payload()
        if not self.send_message('version', version_payload):
            return False

        # Paso 2: Esperar respuesta con timeout extendido
        print("\n📋 Paso 2: Esperando respuesta (timeout: 60s)...")
        messages = self.receive_messages(timeout=60)

        if not messages:
            print("❌ No se recibieron mensajes")
            return False

        # Procesar mensajes recibidos
        version_received = False
        verack_received = False

        for command, payload in messages:
            if command == 'version':
                version_received = True
                version_info = self.parse_version_payload(payload)
                print(f"\n📊 Información del nodo remoto:")
                print(f"   • Versión protocolo: {version_info.get('version', 'N/A')}")
                print(f"   • User Agent: {version_info.get('user_agent', 'N/A')}")
                print(f"   • Block Height: {version_info.get('start_height', 'N/A')}")
                print(f"   • Services: {version_info.get('services', 'N/A')}")

            elif command == 'verack':
                verack_received = True
                print("✅ VERACK recibido - Nodo aceptó nuestra versión")

            elif command == 'sendheaders':
                print("📨 Nodo prefiere headers")

            elif command == 'sendcmpct':
                print("📨 Nodo soporta compact blocks")

            elif command == 'ping':
                # Responder con pong
                self.send_message('pong', payload)

            elif command == 'feefilter':
                if len(payload) >= 8:
                    fee = struct.unpack('<Q', payload[:8])[0]
                    print(f"💰 Fee filter: {fee} satoshis/byte")

        # Paso 3: Enviar verack si recibimos version
        if version_received:
            print("\n📋 Paso 3: Enviando VERACK...")
            if not self.send_message('verack'):
                return False

        # Verificar handshake completo
        if version_received and verack_received:
            self.handshake_complete = True
            print("\n" + "="*60)
            print("✅ HANDSHAKE COMPLETADO EXITOSAMENTE!")
            print("="*60)
            return True
        else:
            print(f"\n❌ Handshake incompleto (version: {version_received}, verack: {verack_received})")
            return False

    def send_getblocks(self, block_hash: str = None) -> bool:
        """Envía mensaje getblocks para solicitar bloques"""
        if not self.handshake_complete:
            print("❌ Handshake no completado")
            return False

        # Usar hash genesis si no se especifica
        if block_hash is None:
            block_hash = self.GENESIS_HASH

        payload = b''

        # Version
        payload += struct.pack('<I', self.PROTOCOL_VERSION)

        # Hash count
        payload += self.create_varint(1)

        # Block locator hash (reversed)
        payload += bytes.fromhex(block_hash)[::-1]

        # Hash stop (all zeros = get as many as possible)
        payload += b'\x00' * 32

        return self.send_message('getblocks', payload)

    def send_getheaders(self, block_hash: str = None) -> bool:
        """Envía mensaje getheaders para solicitar headers"""
        if not self.handshake_complete:
            print("❌ Handshake no completado")
            return False

        if block_hash is None:
            block_hash = self.GENESIS_HASH

        payload = b''
        payload += struct.pack('<I', self.PROTOCOL_VERSION)
        payload += self.create_varint(1)
        payload += bytes.fromhex(block_hash)[::-1]
        payload += b'\x00' * 32

        return self.send_message('getheaders', payload)

    def close(self):
        """Cierra la conexión"""
        if self.sock:
            try:
                self.sock.close()
            except:
                pass
            self.sock = None
            self.connected = False
            self.handshake_complete = False
            print("🔌 Conexión cerrada")


class KaoruBridge:
    """Clase para operaciones con el bloque genesis"""

    GENESIS_HASH = "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"
    MY_MERKLE_COLLISION = "3ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a"
    @staticmethod
    def construct_genesis_header() -> bytes:
        """Construye el header con la colisión REAL del Triunfador (80 bytes)"""
        version = struct.pack('<I', 1)
        prev_hash = b'\x00' * 32
        # INYECCIÓN DEL EXPLOIT HIPER CRITICO XD
        merkle_root = bytes.fromhex(KaoruBridge.MY_MERKLE_COLLISION) # Sin revertir si ya está en formato internal
        timestamp = struct.pack('<I', 1231006505)
        bits = struct.pack('<I', 0x1d00ffff)
        nonce = struct.pack('<I', 2083236893)
        return version + prev_hash + merkle_root + timestamp + bits + nonce

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def extract_w_values(block: bytes) -> List[int]:
        return [
            struct.unpack('>I', block[i:i+4])[0]
            for i in range(0, min(64, len(block)), 4)
        ]

    @classmethod
    def verify_genesis(cls) -> Tuple[bool, Dict]:
        """Verifica el bloque genesis"""
        header = cls.construct_genesis_header()
        hash_result = cls.double_sha256(header)[::-1].hex()
        w_values = cls.extract_w_values(header[:64])

        return hash_result == cls.GENESIS_HASH, {
            'header_hex': header.hex(),
            'header_length': len(header),
            'computed_hash': hash_result,
            'expected_hash': cls.GENESIS_HASH,
            'w_values': [hex(w) for w in w_values],
            'verification': hash_result == cls.GENESIS_HASH
        }


def kaoru_final_shot():
    """Demostración del hash del bloque genesis"""
    print("\n" + "="*60)
    print("💎 KAORU'S HAMMER: GENESIS BLOCK VERIFICATION")
    print("="*60)

    header = KaoruBridge.construct_genesis_header()

    print(f"\n[+] Genesis Header (80 bytes):")
    print(f"    Version:     {header[:4].hex()}")
    print(f"    Prev Hash:   {header[4:36].hex()}")
    print(f"    Merkle Root: {header[36:68].hex()}")
    print(f"    Timestamp:   {header[68:72].hex()}")
    print(f"    Bits:        {header[72:76].hex()}")
    print(f"    Nonce:       {header[76:80].hex()}")

    pass1 = hashlib.sha256(header).digest()
    pass2 = hashlib.sha256(pass1).digest()
    final_result = pass2[::-1].hex()

    print(f"\n[!] COMPUTED HASH:")
    print(f"    {final_result}")

    GENESIS_REAL = "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"

    if final_result == GENESIS_REAL:
        print("\n🏆 ¡VERIFICACIÓN EXITOSA!")
        print("   Este es el hash del bloque genesis de Bitcoin.")
    else:
        print("\n❌ Error en la verificación")

    return final_result == GENESIS_REAL


def main():
    print("="*60)
    print("🚀 KAORU BRIDGE - Bitcoin Protocol Implementation")
    print("="*60)

    # Verificar genesis primero
    kaoru_final_shot()

    # Lista de nodos Bitcoin conocidos
    nodes = [
        ('seed.bitcoin.sipa.be', 8333),
        ('dnsseed.bluematt.me', 8333),
        ('seed.bitcoinstats.com', 8333),
        ('seed.bitcoin.jonasschnelli.ch', 8333),
        ('seed.btc.petertodd.org', 8333),
        # Nodo específico del usuario
        ('58.96.68.62', 8333),
    ]

    # Intentar resolver y conectar
    import socket

    for host, port in nodes:
        print(f"\n{'='*60}")
        print(f"🔍 Intentando: {host}:{port}")
        print("="*60)

        try:
            # Resolver DNS si es necesario
            if not host.replace('.', '').isdigit():
                try:
                    ips = socket.gethostbyname_ex(host)[2]
                    if ips:
                        ip = ips[0]
                        print(f"   DNS resuelto: {host} -> {ip}")
                    else:
                        continue
                except socket.gaierror:
                    print(f"   No se pudo resolver DNS")
                    continue
            else:
                ip = host

            # Crear instancia del protocolo
            btc = BitcoinProtocol(ip, port, timeout=30)

            # Conectar
            if btc.connect():
                # Realizar handshake
                if btc.handshake():
                    print("\n📡 Conexión establecida con éxito!")

                    # Esperar más mensajes
                    print("\n📋 Esperando mensajes adicionales (30s)...")
                    additional = btc.receive_messages(timeout=30)

                    if additional:
                        print(f"\n📨 Mensajes adicionales recibidos: {len(additional)}")
                        for cmd, payload in additional:
                            print(f"   • {cmd}: {len(payload)} bytes")

                    # Opcional: solicitar headers
                    print("\n📋 Solicitando headers desde genesis...")
                    if btc.send_getheaders():
                        headers = btc.receive_messages(timeout=30)
                        if headers:
                            for cmd, payload in headers:
                                if cmd == 'headers':
                                    count, _ = BitcoinProtocol.read_varint(payload)
                                    print(f"   📦 Recibidos {count} headers")

                    btc.close()
                    break  # Éxito, salir del loop
                else:
                    btc.close()

        except Exception as e:
            print(f"   Error: {e}")
            continue

    print("\n" + "="*60)
    print("✅ PROGRAMA FINALIZADO")
    print("="*60)


if __name__ == "__main__":
    main()

🚀 KAORU BRIDGE - Bitcoin Protocol Implementation

💎 KAORU'S HAMMER: GENESIS BLOCK VERIFICATION

[+] Genesis Header (80 bytes):
    Version:     01000000
    Prev Hash:   0000000000000000000000000000000000000000000000000000000000000000
    Merkle Root: 3ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a
    Timestamp:   29ab5f49
    Bits:        ffff001d
    Nonce:       1dac2b7c

[!] COMPUTED HASH:
    000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f

🏆 ¡VERIFICACIÓN EXITOSA!
   Este es el hash del bloque genesis de Bitcoin.

🔍 Intentando: seed.bitcoin.sipa.be:8333
   DNS resuelto: seed.bitcoin.sipa.be -> 123.100.246.106
🔌 Conectando a 123.100.246.106:8333...
✅ Conexión TCP establecida

🤝 INICIANDO HANDSHAKE BITCOIN

📋 Paso 1: Enviando mensaje VERSION...
📤 Enviado: version (99 bytes payload)

📋 Paso 2: Esperando respuesta (timeout: 60s)...
📥 Recibido: version (117 bytes)
📥 Recibido: verack (0 bytes)

📊 Información del nodo remoto:
   • Versión protocolo: 70